# Enunciado

**Objetivo:** 

Desarrollar un programa en Python que permita al usuario buscar expresiones en lunfardo, basándose en descripciones de conceptos o acciones. El sistema deberá comparar la descripción del usuario con las definiciones de un diccionario de lunfardo y devolver las tres palabras cuyas definiciones sean más parecidas a la descripción ingresada.

**Preparación de Datos:**

Extraiga el texto de un diccionario de lunfardo desde un recurso en línea (enlace proporcionado), y organícelo en una tabla de dos columnas:
     - Columna 1: Palabra en lunfardo.
     - Columna 2: Definición de la palabra.

**Desarrollo del Programa:**

Implemente una función `buscar_en_lunfardo(descripcion)` que realice de  las siguientes tareas las que crea útiles:

  1) *Normalización:* Convertir la descripción a minúsculas y eliminar caracteres no alfanuméricos.

  2) *Eliminación de stopwords:* Remover palabras que no aporten significado relevante, utilizando una lista predefinida de stopwords en español.

  3) *Lematización:* Reducir las palabras a su forma base o lema, para facilitar la comparación con las definiciones del diccionario.

  4) *Tokenización:* Dividir la descripción en palabras individuales para su análisis.

  5) *Comparación y Conteo:* Utilizar técnicas básicas de comparación de texto para 
  identificar las tres definiciones en el diccionario que más se asemejen a la descripción ingresada.


# Práctica

In [ ]:
import PyPDF2
import re
import es_core_news_sm

In [ ]:
# Definimos la lista de stopwords en español
stopwords_es = [
    'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 
    'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 
    'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 
    'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 
    'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 
    'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 
    'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 
    'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 
    'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 
    'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 
    'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 
    'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'es'
]

In [ ]:
# Carga del modelo de nlp
nlp = es_core_news_sm.load()  

In [ ]:
def procesar_texto(text: str) -> list[str]:
    """
    Función que a partir de un texto ingresado por argumento, se lo normaliza, elimina stopwords y lematiza. 
    La salida será una lista con sus palabras claves.
    """

    # Eliminación de mayúsculas
    text = text.lower()
    
    # Eliminación puntuación 
    text = re.sub(r'[^\w\s]', '', text)
    
    # Lematización del texto     
    #nlp = es_core_news_sm.load()    
    doc = nlp(text)
    lemmas = [tok.lemma_.lower() for tok in doc]
    
    # Filtramos las palabras para eliminar las stopwords
    palabras_filtradas = [palabra for palabra in lemmas if palabra not in stopwords_es]
    
    return palabras_filtradas

In [ ]:
"""
Celda donde se carga el pdf Lunfardo y se extrae su contenido, siendo la
palabra del lunfardo la clave del diccionario y su definición el valor de la misma.
""" 

# Abre el archivo en modo binario de lectura ('rb')
with open('Lunfardo.pdf', 'rb') as archivo:
    # Crea un objeto PdfFileReader
    lector = PyPDF2.PdfReader(archivo)

# Inicializa una cadena vacía para almacenar el texto
    texto = ''

    # Itera sobre todas las páginas del PDF
    for i in range(1,61):
        # Obtiene la página
        pagina = lector.pages[i]

        # Extrae el texto de la página y lo añade a la cadena de texto
        texto += pagina.extract_text ()

# Diccionario para almacenar las palabras y definiciones
dict_lunfardo = {}

regex = r"(\w+(?:\(se\))?)\s*:\s*(.*?)(?=\n\w+|$)"
resultado = re.findall(regex, texto, re.DOTALL)
dict_lunfardo = {clave: valor.strip() for clave, valor in resultado}

# Limpio y lematizo las definiciones del diccionario
for lunfardo in dict_lunfardo: 
    dict_lunfardo[lunfardo] = procesar_texto(dict_lunfardo[lunfardo])
    try:
        dict_lunfardo[lunfardo].remove(' ')
    except: 
        continue


print(dict_lunfardo)

In [ ]:
def buscador(dict_lunfardo : dict, text: str) -> None:
    """ 
    Funcíon que muestra en pantalla los tres lunfardos que más se asemejan a la definición ingresada por el usuario.
    """
    # Lematizo la definición ingresada
    texto_lemma = procesar_texto(text)

    # Creo un nuevo diccionario, donde la clave será el lunfardo, y su valor el porcentaje de semejanza con la definición.
    porcentajes = {}

    # Itero sobre el diccionario de lunfardos, buscando el que más se adecue a la definición.
    for lunfardo in dict_lunfardo:
        cont_semejanza = 0        
        for palabra in texto_lemma: 
            if palabra in dict_lunfardo[lunfardo]:
                cont_semejanza += 1 
        
        porcentajes[lunfardo] = cont_semejanza
  
    # Ordeno de forma descendiente el diccionario
    porcentajes_ordenado = dict(sorted(porcentajes.items(), key=lambda item: item[1], reverse=True))

    # Muestro en pantalla los 3 lunfardos con mayor semejanza
    tres_primeros = dict(list(porcentajes_ordenado.items())[:3])

    print(tres_primeros)

In [ ]:
# Codigo cliente
buscador(dict_lunfardo, 'Persona que es tonta, con pocas capacidades mentales')
dict_lunfardo